In [1]:
!pip install jsonlines
!pip install sparqlwrapper
!pip install tqdm

import json

from SPARQLWrapper import SPARQLWrapper, JSON
import jsonlines
import pandas as pd
from tqdm import tqdm

In [2]:
kelm_path = "../kelm/kelm_generated_corpus.jsonl"
kelm_entities_path = "../kelm/entities.jsonl"

In [3]:
with open(kelm_path) as json_file:
    json_list = list(json_file)
print(json_list[0])

{"triples": [["Valentin Lavigne", "member of sports team", "FC Lorient"], ["Valentin Lavigne", "FC Lorient", "start time", "01 January 2014"], ["Valentin Lavigne", "FC Lorient", "end time", "01 January 2016"]], "serialized_triples": "Valentin Lavigne member of sports team FC Lorient, FC Lorient end time 01 January 2016, FC Lorient start time 01 January 2014.", "gen_sentence": "Valentin Lavigne played for FC Lorient between 2014 and 2016."}



Todo:

- [] get all person examples about person entities and give gender distribution
- [] get all location examples and determine global distribution

In [13]:
len(json_list)

15628486

# Analyze gender distribution

In [4]:
user_agent = ''
sparql = SPARQLWrapper("https://query.wikidata.org/sparql", agent=user_agent)

def query_entity(wiki_id):
  sparql.setQuery(f"""
  SELECT DISTINCT ?item ?itemLabel ?type ?typeLabel ?gender ?genderLabel
  WHERE 
  {{
    VALUES ?item {{  wd:{wiki_id} }}
    ?item wdt:P31 ?type.
    OPTIONAL {{?item wdt:P21 ?gender.}}
    SERVICE wikibase:label {{ bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en". }}
  }}
  """)
  sparql.setReturnFormat(JSON)
  query_result = sparql.query().convert()
  results = {}
  if len(query_result["results"]["bindings"]) > 0:
    type_label = query_result["results"]["bindings"][0]["typeLabel"]["value"]
    results["type"] = type_label
    if type_label == "human":
      try:
        results["gender"] = query_result["results"]["bindings"][0]["genderLabel"]["value"]
      except:
        results["gender"] = ""
    else:
      results["gender"] = ""
    return results
  else:
    return None


In [5]:
last_written_idx = 62292

In [6]:

with open(kelm_entities_path) as json_file:
    json_list = list(json_file)
    gendered_entities = 0
    for i, json_str in enumerate(tqdm(json_list)):
        if i > last_written_idx:
            item = json.loads(json_str)
            wiki_id = item["id"]
            if wiki_id.startswith("Q"): 
                query_results = query_entity(wiki_id) 
                if query_results is not None:
                    item["type_label"] = query_results["type"]
                    item["gender_label"] = query_results["gender"]
                    with jsonlines.open('gendered_entities.jsonl', mode='a') as writer:
                        writer.write([item])
       

  1%|          | 73121/6902305 [36:19<333:30:31,  5.69it/s] 